In [57]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
words = open('names.txt', 'r').read().splitlines()

In [59]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


In [60]:
# build dataset

block_size = 3
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size 
    for ch in w + '.':
        ix = stoi[ch] # index of current character
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
      
        context = context[1:] + [ix] # sliding window with indexes
 

X = torch.tensor(X)
Y = torch.tensor(Y)
X


... ---> .
emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .


tensor([[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1]])

In [36]:
# lookup table for embed

C = torch.randn((27, 2))
C

tensor([[-8.1166e-01,  7.4129e-01],
        [ 8.3169e-01,  9.8694e-01],
        [-6.9403e-04,  4.6078e-01],
        [ 3.5003e-01, -3.7977e-01],
        [-8.3247e-01,  1.2045e+00],
        [-1.5460e-02,  2.6934e-01],
        [ 3.5078e-01, -1.4617e-01],
        [ 5.5894e-01,  3.0247e-01],
        [-1.2910e+00,  1.1183e+00],
        [ 2.1925e-01, -1.1362e+00],
        [ 8.1331e-01,  1.7408e-02],
        [ 8.9546e-01, -3.9435e-01],
        [-1.0454e-01, -3.0268e-01],
        [ 8.7482e-01,  3.0190e-01],
        [ 3.7067e-01,  1.4021e+00],
        [-5.7123e-02,  1.1384e+00],
        [ 2.6667e+00, -2.5561e-01],
        [-1.8253e+00, -2.5886e-01],
        [-4.8805e-01,  8.7997e-01],
        [ 6.3269e-01,  2.9891e-01],
        [-8.7061e-01, -3.2221e-01],
        [-8.4821e-01, -1.3024e+00],
        [ 1.0188e-01, -2.2998e+00],
        [ 4.6571e-01,  5.6857e-02],
        [-4.5069e-01, -3.0708e+00],
        [-8.9832e-01, -6.2171e-02],
        [-7.6430e-01,  2.1036e+00]])

In [29]:
emb = C[X]
emb

tensor([[[-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [-0.4606, -0.1552]],

        [[-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [-0.4606, -0.1552]],

        [[-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [ 1.6273, -0.6028]],

        [[-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [ 1.6273, -0.6028],
         [ 1.1660,  0.0433]],

        [[-0.4606, -0.1552],
         [ 1.6273, -0.6028],
         [ 1.1660,  0.0433],
         [ 1.1660,  0.0433]],

        [[ 1.6273, -0.6028],
         [ 1.1660,  0.0433],
         [ 1.1660,  0.0433],
         [-2.2991,  0.7821]],

        [[-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [-0.4606, -0.1552]],

        [[-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [-0.4606, -0.1552],
         [-0.6314,  0.7711]],

        [[-0.4606, -0.1552],
         [-0.4606, -0.1552]